In [418]:
import pandas as pd
import numpy as np
import zipfile
import datetime
import seaborn
import warnings
warnings.filterwarnings('ignore')

In [419]:
data_path='C:/Users/user/Documents/Python Scripts/AIMarathon/'

train_zip=zipfile.ZipFile(data_path+'train_offline.csv.zip')    
train=pd.read_csv(train_zip.open('train_offline.csv'))

test_zip=zipfile.ZipFile(data_path+'test_offline.csv.zip')    
test=pd.read_csv(test_zip.open('test_offline.csv')) 

In [420]:
print (train.shape)
print (train.dtypes)
train.head(5)

(1160742, 7)
User_id            int64
Merchant_id        int64
Coupon_id        float64
Discount_rate     object
Distance         float64
Date_received    float64
Date             float64
dtype: object


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [421]:
#確認na值比率
def check_na(df):
    data_na=(df.isnull().sum())/len(df)
    data_na=data_na.drop(data_na[data_na==0].index)
    missing_count=pd.DataFrame({'Missing ratio':data_na})
    display(missing_count)
check_na(train)

,Missing ratio
Coupon_id,0.356473
Discount_rate,0.356473
Distance,0.060156
Date_received,0.356473
Date,0.606537


In [422]:
#把資料轉成日期格式，且因為strptime只能接受str，因此先把nan值判斷成0，之後再把整欄都轉成str
train['Date_received']=train['Date_received'].apply(lambda x: np.int64(x) if np.isnan(x)==False else 0)
train['Date_received']=train['Date_received'].astype('str')
train['Date_received']=train['Date_received'].apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d') if x!='0' else x)

train['Date']=train['Date'].apply(lambda x: np.int64(x) if np.isnan(x)==False else 0)
train['Date']=train['Date'].astype('str')
train['Date']=train['Date'].apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d') if x!='0' else x)

In [423]:
#把train分類成x,y
#取得消費券的有使用的資料是Coupon_id和Date都有數值，且兩者小於等於15天
#先計算出拿到票券後 15天內消費完的人，並把其值設為1，沒有的則全為0

#先把有確實把優惠券使用的人給拉出來
y_target=(train['Date_received']!='0')&(train['Date']!='0')
Use_coupon=train[y_target]
Use_coupon['Use_coupon']=Use_coupon['Date']-Use_coupon['Date_received']
Use_coupon['Use_coupon']=Use_coupon['Use_coupon'].apply(lambda x: 1 if x.days<=15 else 0)
Use_coupon=pd.DataFrame(Use_coupon.loc[:,'Use_coupon'])

#之後合併
train=train.merge(Use_coupon,how='outer',right_index=True,left_index=True)
train['Use_coupon']=train['Use_coupon'].apply(lambda x: 0 if np.isnan(x)==True else x)

print(Use_coupon.shape)
print(train.shape)
train.head(20)

(42936, 1)
(1160742, 8)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Use_coupon
0,1439408,2632,NaN,NaN,0.0,0,2016-02-17 00:00:00,0.0
1,1439408,2632,8591.0,20:1,0.0,2016-02-17 00:00:00,0,0.0
2,1439408,2632,1078.0,20:1,0.0,2016-03-19 00:00:00,0,0.0
3,1832624,3381,7610.0,200:20,0.0,2016-04-29 00:00:00,0,0.0
4,2029232,3381,11951.0,200:20,1.0,2016-01-29 00:00:00,0,0.0
5,2223968,3381,9776.0,10:5,2.0,2016-01-29 00:00:00,0,0.0
6,73611,2099,12034.0,100:10,NaN,2016-02-07 00:00:00,0,0.0
7,163606,1569,5054.0,200:30,10.0,2016-04-21 00:00:00,0,0.0
8,3273056,4833,7802.0,200:20,10.0,2016-01-30 00:00:00,0,0.0
9,94107,3381,7610.0,200:20,2.0,2016-04-12 00:00:00,0,0.0


In [680]:
#去除沒有收到coupon的人 (因為submitssion只要繳交有收到coupon)
x_train=train[pd.isna(train['Coupon_id'])==False]
y_train=train[['Use_coupon','Date_received']]
y_train=y_train[y_train['Date_received']!='0']
y_train=y_train.drop(['Date_received'],axis=1)

check_na(x_train)

In [682]:
#補缺值
#Distance屬於類別型，非連續型數值，因此都採用眾數的方式填補
x_train['Distance']=x_train['Distance'].fillna(x_train['Distance'].mode()[0])
check_na(x_train)

In [693]:
#整理Feature

#Discount
#把Discount_rate轉成rate，並分門別類，其他的也要轉成數值
def Discount_type(row):
    if ':' in row:
        return 1
    else:
        return 0
    
def Turn_to_rate(row):
    if ':' in  row:
        new_row=row.split(':')
        return 1.0 - float(new_row[1])/float(new_row[0])
    else:
        return float(row)

def Discount_condition(row):
    if ':' in row:
        new_row=row.split(':')
        return float (new_row[1])
    else:
        return 0
    
x_train['Discount_type']=x_train['Discount_rate'].apply(Discount_type)
x_train['New_Discount_rate']=x_train['Discount_rate'].apply(Turn_to_rate)
x_train['Discount_condition']=x_train['Discount_rate'].apply(Discount_condition)

#假設Merchant_id和Coupon_id以100為單位
x_train['New_Merchant_id']=(x_train['Merchant_id']/100).astype('int')
x_train['New_Coupon_id']=(x_train['Coupon_id']/100).astype('int')

#Date_received
#假設收到coupon與發薪日有關 
def new_date_received(row):
    day=int(datetime.datetime.strftime(row, '%d'))
    if  day <11:
        return 0
    elif day >20:
        return 2
    else:
        return 1

x_train['New_date_received']=x_train['Date_received'].apply(new_date_received)


x_train.head(10)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Use_coupon,Discount_type,New_Discount_rate,Discount_condition,New_Merchant_id,New_Coupon_id,New_date_received
1,1439408,2632,8591.0,20:1,0.0,2016-02-17 00:00:00,0,0.0,1,0.95,1.0,26,85,1
2,1439408,2632,1078.0,20:1,0.0,2016-03-19 00:00:00,0,0.0,1,0.95,1.0,26,10,1
3,1832624,3381,7610.0,200:20,0.0,2016-04-29 00:00:00,0,0.0,1,0.90,20.0,33,76,2
4,2029232,3381,11951.0,200:20,1.0,2016-01-29 00:00:00,0,0.0,1,0.90,20.0,33,119,2
5,2223968,3381,9776.0,10:5,2.0,2016-01-29 00:00:00,0,0.0,1,0.50,5.0,33,97,2
6,73611,2099,12034.0,100:10,0.0,2016-02-07 00:00:00,0,0.0,1,0.90,10.0,20,120,0
7,163606,1569,5054.0,200:30,10.0,2016-04-21 00:00:00,0,0.0,1,0.85,30.0,15,50,2
8,3273056,4833,7802.0,200:20,10.0,2016-01-30 00:00:00,0,0.0,1,0.90,20.0,48,78,2
9,94107,3381,7610.0,200:20,2.0,2016-04-12 00:00:00,0,0.0,1,0.90,20.0,33,76,1
11,253750,8390,7531.0,20:5,0.0,2016-03-27 00:00:00,0,0.0,1,0.75,5.0,83,75,2


In [700]:
#一樣整理test
test=test.sort_values(by=['User_id','Merchant_id','Date_received'])
x_test=test[pd.isna(test['Date_received'])==False]

#
#把資料轉成日期格式，且因為strptime只能接受str，把整欄都轉成str
x_test['Date_received']=x_test['Date_received'].astype('int')
x_test['Date_received']=x_test['Date_received'].astype('str')
x_test['Date_received']=x_test['Date_received'].apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d'))

#補缺值
#Distance屬於類別型，非連續型數值，因此都採用眾數的方式填補
x_test['Distance']=x_test['Distance'].fillna(x_test['Distance'].mode()[0])

#處理Discount_rate，用label encodeing
x_test['Discount_type']=x_test['Discount_rate'].apply(Discount_type)
x_test['New_Discount_rate']=x_test['Discount_rate'].apply(Turn_to_rate)
x_test['Discount_condition']=x_test['Discount_rate'].apply(Discount_condition)

#假設Merchant_id和Coupon_id以100為單位有關連性
x_test['New_Merchant_id']=(x_test['Merchant_id']/100).astype('int')
x_test['New_Coupon_id']=(x_test['Coupon_id']/100).astype('int')

#Date_received
#假設收到coupon與發薪日有關 
x_test['New_date_received']=x_test['Date_received'].apply(new_date_received)



In [701]:
print (check_na(x_train))
print (check_na(x_test))

,Missing ratio


None


,Missing ratio


None


In [702]:
x_test.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Discount_type', 'New_Discount_rate',
       'Discount_condition', 'New_Merchant_id', 'New_Coupon_id',
       'New_date_received'],
      dtype='object')

In [715]:
Feature=[ 'Distance', 'Discount_type', 'New_Discount_rate',
       'Discount_condition',
       'New_date_received']
x_train=x_train[Feature]
x_test=x_test[Feature]
#使用梯度提升數分類
#from sklearn.ensemble import GradientBoostingClassifier
clf=GradientBoostingClassifier()
clf.fit(x_train,y_train)
y_test=clf.predict_proba(x_test)


In [712]:
#做出uid
uid=test[pd.isna(test['Date_received'])==False]
uid_columns=['User_id','Coupon_id','Date_received']
uid=uid[uid_columns]
uid['Date_received']=uid['Date_received'].astype('int')
uid['Coupon_id']=uid['Coupon_id'].astype('int')
uid=uid.astype('str')
uid['two_id']=uid['User_id'].str.cat(uid['Coupon_id'],sep='_')
uid['uid']=uid['two_id'].str.cat(uid['Date_received'],sep='_')
uid=pd.DataFrame(uid['uid'])
uid=uid.reset_index(drop=True)
uid.head(5)

,uid
0,4_2902_20160607
1,165_7571_20160525
2,166_9261_20160525
3,215_8944_20160524
4,236_11002_20160528


In [716]:
final=pd.DataFrame(y_test)
final.columns=['x','label']
final=final.drop('x',axis=1)
submission=uid.merge(final,how='inner',right_index=True,left_index=True)

In [717]:
submission=submission.groupby('uid',as_index=False).mean()
submission.to_csv('submission.csv',index=False)

![Submission](https://lh3.googleusercontent.com/RH0qdqfd8r9jUXWRAdHCBpq-b_3EZD_aFfWhyViwe9qJ-uu_SuaWuvJTbjJ7GdGqg11z1jg3P41TgBk-0VaEr5nu5ZsZ7GQjFpgRPIy8yrjxjTfMRVzEZsyi7ih-s-2ouYEFwEKE5Le-kj-cfolSpc0dfBpG1-aCLRyTct5sV4WZ4uDlOS6VPSEnTNavDNa30n3TsP23DCTr13HsC9x6FCiZqNKsL8lQ9m2yEtX3zHSo1x5aOLiBCl0Uj7nsZEGkbzU7HsqSUQACvLn7ws41fxeCC3d9tv2fOKuBTJqV84gzAHRs2q4NfwMYVATwSSpHm2htfaQW2jQAOaHUSC-BSoCIzsm8RNJsIoNb0HrvGvw9bEOSNJEGZiodie-vJD40tBZqSgoBhYygrzSwJwcthIKcd7jVbKW4OKtY5wjSYQ8dsIWEAG9dpaXzPQlcMtGhTtqr3y61BgkMnwJeL4OGzIS4JtyBisyjoYQTm4RPXLowhpIbQnRnw1LZly3tgcl1WW_joCJ8Sp_dvY39zcv1UXcOCfrJTCP2adqgLQP1GFwFsB6rh9vsgjacXZzuR9pixjkZg2CIabUYNcakw79gCh2tJ_d52W-y_ARRggvSifAu7bCKUav0Hv-kU_WNNX0O_ml62L2AOw8L2KcTP80pV1I283BeGesCFgBTac8dxaOoUMgVyXA5AYA7snlswaTR5Olgk7CgIM6e4ppbs8dAQUEX6A=w2565-h1443-no)